In [ ]:
%matplotlib inline

# Overview Tasmania example of the AWRA-CMS functionality

The purpose of this notebook is to give you an overview of the AWRA-CMS functionality for a given example. 

We have previously given a description of the AWRA-L model in python in the notebook: [1.3_AWRA-L_deconstructed_-_Python_version_of_AWRA-L]
[1.3_AWRA-L_deconstructed_-_Python_version_of_AWRA-L]: /tree/Training/Basics/1.3_AWRA-L_deconstructed_-_Python_version_of_AWRA-L.ipynb. 

A quick refresher of the components of the AWRA MS:
- **Simulation** *Module:* **awrams.simulation** <br> A specified run of the AWRA model over a time period and spatial extent
- **Calibration** *Module:* **awrams.calibration** <br> Using an optimisation algorithm and an objective function to obtain the optimum set of AWRA-L model parameters for a given time period, extent, etc.
- **Visualisation** *Module:* **awrams.visualisation**<br> A figure showing AWRA-L inputs and outputs as maps and time-series
- **Benchmarking** *Module:* **awrams.benchmarking**<br> Validating model outputs compared to observed data or other model simulations
- **Extraction** <br> Extraction of point or catchment average data from the AWRA-L grids
- **Utilities** *Module:* **awrams.utils** <br> Functionality that is required in more than one of the AWRA MS components 


Let's also define some key terms:
- **Nodegraph** 
    - Node - a single input component 
    - Graph - the order that these input components are defined
    - In the AWRA MS we have an 'input nodegraph' and an 'output nodegraph'
- **Forcing data** <br> 
    Gridded inputs to our model (i.e. precipitation, solar exposure, max temperature, min temperature
- **Climatology** <br>
    Mean climate state for each day/month of the year used by the AWRA MS to infill gaps in the forcing data
- **Initial states** <br>
    The AWRA-L model has a number of states that change from one time-step to the next (i.e. mleaf, s0, ss, sd, sg, sr). Without specifying these the default settings assume all soil and ground stores are half-full.
- **Spatial extent** <br> This is the area that we will undertake a calibration or simulation or visualisation   

For more definition of the AWRA-CMS see the user guide: [AWRA-CMS User guide ]
[AWRA-CMS User guide ]: /tree/Training/AWRA_Landscape_Community_Modelling_System_User_Guide_2017.pdf

**Tasmania example:**<br>
Rather than using a simulation from the whole continent, we would like to undertake a model calibration using a set of five Tasmanian catchments only (IDs: 306119, 308145, 312061, 318150, 318852). Once we have calibrated the model we will simulate over Tasmania and visualise these results.

The notebook follows a typical workflow of calibration, simulation, benchmarking and visualisation with the following steps: 
1. Import required libraries
2. Configure model
3. Calibrate the AWRA-L model to the Tasmanian catchments
4. Simulation over Tasmania
5. Benchmark to standard AWRA-L calibration
6. Visualisation of results

*After this example overview session we will go through all of these steps in much greater detail during the upcoming sessions.*

***

## 1. Import required libraries

In [ ]:
# Get os path
from os.path import join

# import various python modules
import pandas as pd
import h5py
from matplotlib import pyplot as plt

# Import AWRA MS utilities
from awrams.utils import config_manager, extents, gis
from awrams.utils import datetools as dt
from awrams.utils.gis import ShapefileDB
from awrams.utils.nodegraph import nodes, graph

# Import AWRA MS calibration
from awrams.calibration import cluster, support
from awrams.calibration.launch_calibration import run_from_pickle
from awrams.calibration.objectives import test_objectives as tobj
from awrams.calibration.optimizers import sce

In [ ]:
# Get the base data path from the system profile
sys_profile = config_manager.get_system_profile()
sys_settings = sys_profile.get_settings()

base_data_path = sys_settings['DATA_PATHS']['BASE_DATA']

### Questions?

## 2. Configure model

In [ ]:
# Instantiate the AWRA-L model with default model settings
model_profile = config_manager.get_model_profile('awral', 'v6_default')
awral = model_profile.get_model()

In [ ]:
#Specify running and calibration period
run_period = dt.dates('2009 - 2011')
eval_period = dt.dates('2010 - 2011')

### 2.1 Specify calibration catchments

In [ ]:
# Set list of catchment ids
id_list = ['306119', '308145', '312061', '318150', '318852']

In [ ]:
# Get the default extent of the calibration (This is the whole of Australia capturing all catchments)
catchments_shapefile = join(base_data_path, 'spatial/shapefiles/Final_list_all_attributes.shp')
calvalshapefile = ShapefileDB(catchments_shapefile)
def_extent = extents.get_default_extent() 

## Create a dict with multiple Tasmanian catchment extents
cal_dict = {}
for catchment_id in id_list:
    cal_dict[catchment_id] = calvalshapefile.get_extent_by_field(
            'StationID', catchment_id.zfill(6), parent_extent=def_extent)

cal_dict

In [ ]:
# Visualise the Tasmanian catchment locations (Note that some of these are quite small to see)
import awrams.visualisation.results as res
import awrams.visualisation.vis as vis

vis.show_extent(cal_dict['312061'],def_extent)
vis.show_extent(cal_dict['318150'],def_extent)
vis.show_extent(cal_dict['306119'],def_extent)
vis.show_extent(cal_dict['318852'],def_extent)
vis.show_extent(cal_dict['308145'],def_extent)

In [ ]:
# Specify the location of the observed data
observations = dict(qtot=base_data_path + '/observations/runoff/awrams_v5_cal_qobs.csv')

### 2.2 Specify input mapping (input nodegraph)

In [ ]:
input_map = model_profile.get_input_mapping()

In [ ]:
def insert_solar_climatology(imap):
    from awrams.utils.nodegraph import nodes

    climatologies = {
        'solar': (
            base_data_path + '/training/simulation/climatology/Rad_1990_2009.nc', 'solar_exposure_day'
        )
    }

    imap = imap.copy()

    imap['solar_f_orig'] = imap['solar_f'] #'Move' the forcing node to a new name
    imap['solar_climatology_f'] = nodes.monthly_climatology(*climatologies['solar']) # Loads monthly climatology from default file

    # Replace 'solar_f' with infilled data
    # This ensures that any other nodes in the graph who use solar_f as input will automatically receive the infilled data
    # nodes.gap_filler takes the first argument as 'gappy' data, and in infills with data from the second argument
    imap['solar_f'] = nodes.gap_filler('solar_f_orig', 'solar_climatology_f')
    
    return imap

In [ ]:
input_map = insert_solar_climatology(input_map)

In [ ]:
input_map

### Questions?

## 3. Calibrate the AWRA-L model to the Tasmanian catchments

In [ ]:
#Select optimisation algorithm
evolver_spec = support.EvolverSpec(sce.CCEvolver,
                                   evolver_run_args=dict(n_offspring=1, n_evolutions=5, elitism=2.0))

# Please note that these specifications are designed to reduce runtime for demo purposes
optimizer_spec = support.OptimizerSpec(sce.ShuffledOptimizer,
                                       evolver_spec=evolver_spec,
                                       n_complexes=3, # (Initial) number of complexes
                                       max_nsni=300, # Maximum number of shuffles without improvement
                                       min_imp = 0.1, # Minimum proportion of improvement to occure over max_nsni
                                       min_complexes=1, # Mininum number of complexes
                                       max_eval=1000) # Maximum model evaluations before terminating

#optimizer_spec = OptimizerSpec(SobolOptimizer,threshold = 0.005,max_eval =10000)

In [ ]:
#Import objective function
local_objfspec = support.ObjectiveFunctionSpec(tobj.TestLocalSingle)  # this function to load up the ObjFunc comes from awrams.calibration.support
global_objfspec = tobj.TestGlobalSingle

In [ ]:
objective_spec = support.ObjectiveSpec(global_objfspec, local_objfspec, observations, eval_period)

In [ ]:
# Build spec dict
# Get the input mapping and model
node_mapping = input_map
model = awral#callable_to_funcspec(awral)

In [ ]:
# Create the calibration specification dictionary

'''
User specifiable calibration description
'''
cal_spec = {}
cal_spec['optimizer_spec'] = optimizer_spec
cal_spec['objective_spec'] = objective_spec
cal_spec['extent_map'] = cal_dict
cal_spec['run_period'] = run_period
cal_spec['eval_period'] = eval_period
cal_spec['model'] = model
cal_spec['node_mapping'] = node_mapping
#cal_spec['logfile'] = '/short/er4/dss548/calibration_res.h5'
cal_spec['logfile'] = './tas_cal.h5'

In [ ]:
# Save a file containing all the calibration specifications
nnodes = 1
ncores = 4
cluster.build_pickle_from_spec(cal_spec, ncores, nnodes, 'tas_cal.pkl')

In [ ]:
print("Depending on the specifications of your machine the calibration will run for some time")
cal = run_from_pickle('./tas_cal.pkl') # run the calibration

In [ ]:
# Obtain the best (minimum score) parameter set
cr = support.CalibrationResults('./tas_cal.h5')

In [ ]:
best_params = cr.get_best_paramset()
new_params = pd.DataFrame(best_params, columns=['value'])
new_params['MemberName'] = new_params.index

In [ ]:
# Run the model with new parameter set. Change the parameter set in the input map
def change_model_parameters(imap, param_df):
    """
    the intent of this function is to replace the values in imap (your current configuration) 
    by values in a new parameter dataframe (param_df)
    """
    
    import pandas as pd
    
    variable_names = param_df['MemberName']
    variable_value = param_df['value']
      
    for k,v in zip(variable_names, variable_value):
        imap[k.lower()].args['value'] = v
        
change_model_parameters(input_map, new_params)

### Questions?

## 4. Simulate over Tasmania

In [ ]:
#Load up a  server simulator to run the model to write outputs out
from awrams.simulation import server

In [ ]:
# Confirm output settings
omap = awral.get_output_mapping()

def build_output_mapping(output_map, outpath):
    from awrams.utils.nodegraph import nodes
   
    FILE_OUTPUT_VARS = ['s0', 'ss', 'sd', 'qtot', 'etot']
    
    for f in FILE_OUTPUT_VARS:
            output_map[f + '_ncsave'] = nodes.write_to_annual_ncfile(outpath, f)

    return output_map

In [ ]:
# Set simulation output path
outpath = './_results_tas_cal/'
omap = build_output_mapping(omap, outpath)

In [ ]:
from awrams.utils import datetools as dt
period = dt.dates('2010 - 2011')

In [ ]:
## The functionality to model multiple extents at a time is coming. In the meantime, either:
## model 1 catchment at a time or model the entire bounding box..ie. Tasmania coords[-39.5:-44,143.5:149]
sim = server.SimulationServer(awral)
sim.run(input_map, omap, period, def_extent.icoords[-39.5:-44, 143.5:149])
# You may get a netCDF error, if so you may need to delete the previous simulation "./_results_tas_cal/" and then try again

In [ ]:
# Extract outputs into csv's for each catchment
import os
import pandas as pd
from awrams.utils import extents
from awrams.utils.gis import ShapefileDB
from awrams.utils.io.data_mapping import SplitFileManager
from awrams.utils.processing.extract import extract_from_filemanager

catchments_shapefile = join(base_data_path, 'spatial/shapefiles/Final_list_all_attributes.shp')
catchments = ShapefileDB(catchments_shapefile)

var_name = 'qtot'
model_data_path = os.getcwd() + '/_results_tas_cal/'
period = dt.dates('2010 - 2011')

pattern = model_data_path + '/%s*' % var_name
sfm = SplitFileManager.open_existing(model_data_path, pattern, var_name)
georef = sfm.get_extent()
    
df = extract_from_filemanager(sfm, cal_dict, period)
df

In [ ]:
df.plot()

In [ ]:
df.to_csv('./tas_cal_qtot.csv')

### Questions?

## 5. Benchmark to observed data and a standard AWRA-L calibration

In [ ]:
# Import Benchmarking module
from awrams.benchmarking.benchmark import Benchmark
obs_csv = base_data_path + '/observations/runoff/awrams_v5_cal_qobs.csv'

In [ ]:
# Define Benchmarking object
q = Benchmark("QObs", "runoff")

# Specify benchmarking period
q.period = dt.dates("2010 -2011")

In [ ]:
# Add observations and catchment subset [the id list needs to be present in the column names of the observation file]
q.load(obs_csv, id_list=id_list)

In [ ]:
## Add model outputs
## Default AWRAL calibration
csv_data = base_data_path + '/training/benchmarking/runoff/AWRAMSI_v5QES_AWRAL_qtot_avg.csv'
q.add_model("AWRAMSI_v5_0_AWRAL", data_csv=csv_data)

## Regional Tasmanian calibration
csv_data = './tas_cal_qtot.csv'
q.add_model("Tas_Cal", data_csv=csv_data)

In [ ]:
# View benchmarking statistics from both the standard AWRA calibration (AWRAMSI_v5_0_AWRAL) 
# and our Tasmanian calibration (Tas_Cal)
q.benchmark.stat_percentiles('nse', freq='d')

In [ ]:
q.benchmark.stat(statistic='nse')

In [ ]:
q.benchmark.stat() 

In [ ]:
# plot a timeseries of observed (black) and predicted streamflow from both the standard 
# AWRA calibration (AWRAMSI_v5_0_AWRAL) and our Tasmanian calibration (Tas_Cal)
p = q.benchmark.plot_timeseries('306119', ylim=[0,50], freq = 'd')

### Questions?

## 6. Visualise over the spatial extent (Tasmania)

In [ ]:
# Import AWRA MS modules
import awrams.visualisation.vis as vis
import awrams.visualisation.results as res
import awrams.utils.extents as extents

In [ ]:
# Load the results from the simulation
results = res.load_results('./_results_tas_cal')

In [ ]:
# Here we visualise all of the outputs, on a single day, over the entire spatial extent of the simulation
results[:,'march 2010',:].spatial()

### Questions?

## That finishes a quick example of some of the AWRA Community Modelling system functionality. Throughout the rest of the course we will focus on individual elements in greater detail to highlight the flexibility of the modeling system for various applications.

## Any other questions?